<a href="https://colab.research.google.com/github/JuanAquino22/project_ia/blob/main/projectIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Transformación de Oraciones en Guaraní con RAG

## Proyecto Final - PLN e IA

**Objetivo:** Implementar un sistema capaz de transformar oraciones en guaraní según reglas gramaticales específicas, comparando el rendimiento de:
- 2 modelos de LLM (GPT-3.5 Turbo vs Claude 3.5 Sonnet)
- 2 estrategias: **Sin RAG** (conocimiento del modelo) vs **Con RAG** (recuperación de documentación gramatical)

## Dataset: AmericasNLP 2025
- **Task:** Transformación de oraciones en guaraní
- **Input:** `Source` (oración base) + `Change` (tipo de transformación)
- **Output:** `Target` (oración transformada)

**Ejemplo:**
```
Source: "Ore ndorombyai kuri"
Change: "TYPE:AFF" (convertir a afirmativo)
Target: "Ore rombyai kuri"
```

## Metodología
1. Cargar dataset AmericasNLP (train/dev/test)
2. Construir base de conocimiento (RAG) con gramática guaraní
3. Implementar sistema de transformación con y sin RAG
4. Evaluar con métricas objetivas (BLEU, accuracy, etc.)
5. Comparar resultados

## 1. Instalación de Dependencias

In [5]:
import warnings
warnings.filterwarnings("ignore")

# Instalación de librerías principales
!pip install -q -U \
  langchain langchain-core langchain-community langchain-text-splitters \
  sentence-transformers faiss-cpu \
  requests pymupdf pandas sacrebleu nltk \
  huggingface_hub tqdm

print("✓ Dependencias instaladas correctamente")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.1/102.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.0/475.0 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 114.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 144.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.

## 2. Importaciones y Configuración

In [6]:
import os
import json
import time
import pandas as pd
import fitz  # PyMuPDF
from typing import List, Dict, Any, Tuple

from tqdm import tqdm

# LangChain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document

# Métricas
from sacrebleu.metrics import BLEU

print("✓ Importaciones completadas")


✓ Importaciones completadas


## 3. Configuración de API Keys

In [7]:
import os

from google.colab import userdata

# OpenRouter
os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER")

# Hugging Face
hf_token = userdata.get("HF_TOKEN")
if hf_token:
    os.environ["HF_TOKEN"] = hf_token
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token

print("OpenRouter:", bool(os.environ.get("OPENROUTER_API_KEY")))
print("HF_TOKEN:", bool(os.environ.get("HF_TOKEN")))


OpenRouter: True
HF_TOKEN: True


## 4. Descarga del Dataset AmericasNLP

Dataset oficial: https://github.com/AmericasNLP/americasnlp2025/tree/main/ST2_EducationalMaterials/data

In [8]:
# URLs del dataset
BASE_URL = "https://raw.githubusercontent.com/AmericasNLP/americasnlp2025/main/ST2_EducationalMaterials/data/"

datasets_urls = {
    "train": f"{BASE_URL}guarani-train.tsv",
    "dev": f"{BASE_URL}guarani-dev.tsv",
    "test": f"{BASE_URL}guarani-test.tsv"
}

def load_dataset(url: str) -> pd.DataFrame:
    """Carga el dataset desde URL"""
    response = requests.get(url)
    response.raise_for_status()

    # Guardar temporalmente
    filename = url.split("/")[-1]
    with open(filename, "w", encoding="utf-8") as f:
        f.write(response.text)

    # Leer con pandas
    df = pd.read_csv(filename, sep="\t")
    return df

# Cargar datasets
print("Descargando datasets...\n")
datasets = {}

for split, url in datasets_urls.items():
    try:
        df = load_dataset(url)
        datasets[split] = df
        print(f"✓ {split.upper()}: {len(df)} ejemplos")
        print(f"  Columnas: {list(df.columns)}")
    except Exception as e:
        print(f"❌ Error cargando {split}: {e}")

# Vista previa
print("\n" + "="*60)
print("EJEMPLO DEL DATASET (primeras 3 filas de DEV):")
print("="*60)
print(datasets["dev"].head(3))

Descargando datasets...

✓ TRAIN: 178 ejemplos
  Columnas: ['ID', 'Source', 'Change', 'Target']
✓ DEV: 79 ejemplos
  Columnas: ['ID', 'Source', 'Change', 'Target']
✓ TEST: 364 ejemplos
  Columnas: ['ID', 'Source', 'Change', 'Target']

EJEMPLO DEL DATASET (primeras 3 filas de DEV):
            ID               Source           Change                Target
0  Guarani0232  Ore ndorombyai kuri         TYPE:AFF      Ore rombyai kuri
1  Guarani0233  Ore ndorombyai kuri    TENSE:FUT_SIM      Ore ndorombyaita
2  Guarani0234  Ore ndorombyai kuri  PERSON:1_PL_INC  Ñande nañambyai kuri


## 5. Carga de Gramática Guaraní (Base de Conocimiento para RAG)

**Nota:** Sube el archivo `Gramática guaraní.pdf` a Colab o usa Google Drive.

In [9]:
from google.colab import drive
drive.mount('/content/drive')

# Rutas de los documentos en tu Drive
PDF_GRAMMAR = "/content/Gramática guaraní.pdf"
PDF_DICT    = "/content/Diccionario Guaraní-Español  Español-Guaraní.pdf"

print("✓ Archivos cargados desde Google Drive")
print("Gramática:", PDF_GRAMMAR)
print("Diccionario:", PDF_DICT)


Mounted at /content/drive
✓ Archivos cargados desde Google Drive
Gramática: /content/Gramática guaraní.pdf
Diccionario: /content/Diccionario Guaraní-Español  Español-Guaraní.pdf


In [10]:
def extract_text_from_pdf(pdf_path: str) -> str:
    """Extrae texto de un PDF"""
    doc = fitz.open(pdf_path)
    text = ""

    print(f"📘 Extrayendo texto de: {pdf_path}")
    print(f"   Total de páginas: {len(doc)}")

    for page_num, page in enumerate(doc, start=1):
        text += page.get_text()

        if page_num % 10 == 0:
            print(f"   • Procesadas: {page_num} páginas")

    doc.close()
    print(f"✓ Extracción completada")
    return text.strip()


# Extraer texto de gramática y diccionario
grammar_text = extract_text_from_pdf(PDF_GRAMMAR)
dict_text    = extract_text_from_pdf(PDF_DICT)

# Unir ambos para el RAG
combined_text = grammar_text + "\n\n" + dict_text

print("\nMUESTRA DEL TEXTO (primeros 500 caracteres):")
print(combined_text[:500] + "...")


📘 Extrayendo texto de: /content/Gramática guaraní.pdf
   Total de páginas: 260
   • Procesadas: 10 páginas
   • Procesadas: 20 páginas
   • Procesadas: 30 páginas
   • Procesadas: 40 páginas
   • Procesadas: 50 páginas
   • Procesadas: 60 páginas
   • Procesadas: 70 páginas
   • Procesadas: 80 páginas
   • Procesadas: 90 páginas
   • Procesadas: 100 páginas
   • Procesadas: 110 páginas
   • Procesadas: 120 páginas
   • Procesadas: 130 páginas
   • Procesadas: 140 páginas
   • Procesadas: 150 páginas
   • Procesadas: 160 páginas
   • Procesadas: 170 páginas
   • Procesadas: 180 páginas
   • Procesadas: 190 páginas
   • Procesadas: 200 páginas
   • Procesadas: 210 páginas
   • Procesadas: 220 páginas
   • Procesadas: 230 páginas
   • Procesadas: 240 páginas
   • Procesadas: 250 páginas
   • Procesadas: 260 páginas
✓ Extracción completada
📘 Extrayendo texto de: /content/Diccionario Guaraní-Español  Español-Guaraní.pdf
   Total de páginas: 218
   • Procesadas: 10 páginas
   • Procesadas: 2

## 6. Construcción del Vector Store (RAG)

In [11]:
# Unir textos de gramática y diccionario
text_corpus = grammar_text + "\n\n" + dict_text

# Dividir texto en chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

print("📚 Dividiendo texto en chunks...")
text_chunks = text_splitter.split_text(text_corpus)

# Crear documentos para el vectorstore
documents = [
    Document(
        page_content=chunk,
        metadata={"source": "gramatica+diccionario", "chunk_id": idx}
    )
    for idx, chunk in enumerate(text_chunks)
]

print(f"✓ Creados {len(documents)} chunks")
print(f"  Tamaño promedio: {sum(len(c) for c in text_chunks) / len(text_chunks):.0f} caracteres")


📚 Dividiendo texto en chunks...
✓ Creados 974 chunks
  Tamaño promedio: 977 caracteres


In [12]:
# Cargar modelo de embeddings
print("🔄 Cargando modelo de embeddings...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)
print("✓ Modelo de embeddings listo")

🔄 Cargando modelo de embeddings...


/tmp/ipython-input-3963190336.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ Modelo de embeddings listo


In [13]:
# Crear vector store con FAISS
print("💾 Creando vector store...")
vectorstore = FAISS.from_documents(documents, embeddings)

# Configurar retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

print("✓ Vector store creado")
print("✓ Retriever configurado (k=3)")


💾 Creando vector store...
✓ Vector store creado
✓ Retriever configurado (k=3)


## 7. Wrapper para OpenRouter (GPT-3.5 y Claude 3.5)

In [14]:
class OpenRouterLLM:
    """Wrapper para usar modelos via OpenRouter"""

    def __init__(self, model_name: str, api_key: str):
        self.model_name = model_name
        self.api_key = api_key
        self.api_url = "https://openrouter.ai/api/v1/chat/completions"

    def generate(self, prompt: str, max_tokens: int = 200, temperature: float = 0.3) -> str:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://colab.research.google.com/",
            "X-Title": "guarani-transformation"
        }

        data = {
            "model": self.model_name,
            "messages": [
                {"role": "system", "content": "Eres un modelo que transforma oraciones en guaraní según reglas dadas."},
                {"role": "user", "content": prompt}
            ],
            "max_tokens": max_tokens,
            "temperature": temperature
        }

        try:
            response = requests.post(self.api_url, headers=headers, json=data, timeout=60)
            if response.status_code == 200:
                out = response.json()["choices"][0]["message"]["content"]
                return out if isinstance(out, str) else str(out)
            else:
                return f"Error {response.status_code}: {response.text}"
        except Exception as e:
            return f"Error: {str(e)}"


# Configurar modelos
api_key = os.environ.get("OPENROUTER_API_KEY")

model_gpt35 = OpenRouterLLM("openai/gpt-3.5-turbo", api_key)
model_claude = OpenRouterLLM("anthropic/claude-3.5-sonnet", api_key)

print("✓ Modelos configurados:")
print("  - GPT-3.5 Turbo (OpenAI)")
print("  - Claude 3.5 Sonnet (Anthropic)")


✓ Modelos configurados:
  - GPT-3.5 Turbo (OpenAI)
  - Claude 3.5 Sonnet (Anthropic)


## 8. Sistema de Transformación de Oraciones

In [15]:
class GuaraniTransformationSystem:
    """Sistema de transformación de oraciones en guaraní"""

    def __init__(self, llm, retriever=None):
        self.llm = llm
        self.retriever = retriever

    def transform_without_rag(self, source: str, change: str) -> str:
        """Transformación SIN RAG"""
        prompt = f"""
Transforma la siguiente oración en guaraní según la regla indicada.

Oración original: {source}
Regla de transformación: {change}

Responde ÚNICAMENTE con la oración transformada en guaraní:
"""
        return self.llm.generate(prompt)

    def transform_with_rag(self, source: str, change: str) -> str:
        """Transformación CON RAG"""

        # RAG: buscar contexto útil según la regla
        query = f"{change} {source} reglas gramática guaraní"
        docs = self.retriever.invoke(query)
        context = "\n\n".join([d.page_content[:500] for d in docs[:3]])

        prompt = f"""
Usa el siguiente contexto gramatical en guaraní para realizar la transformación:

CONTEXTO:
{context}

TAREA:
Transformar la oración según la regla dada.

Oración original: {source}
Regla: {change}

Responde ÚNICAMENTE con la oración transformada en guaraní:
"""
        return self.llm.generate(prompt)

    def evaluate_dataset(self, df: pd.DataFrame, use_rag: bool = False) -> List[Dict]:
        """Evalúa el sistema en un dataset completo"""
        results = []

        for idx, row in df.iterrows():
            source = row["Source"]
            change = row["Change"]
            target = row["Target"]

            # Generar predicción
            if use_rag:
                prediction = self.transform_with_rag(source, change)
            else:
                prediction = self.transform_without_rag(source, change)

            pred_clean = prediction.strip().replace('"', '').lower()
            targ_clean = target.strip().replace('"', '').lower()

            results.append({
                "id": row["ID"],
                "source": source,
                "change": change,
                "target": target,
                "prediction": prediction,
                "correct": pred_clean == targ_clean
            })

            # evitar saturar OpenRouter
            time.sleep(0.4)

        return results

print("✓ Sistema de transformación implementado")


✓ Sistema de transformación implementado


## 9. Evaluación de Modelos

Se evaluarán ambos modelos (GPT-3.5 y Claude 3.5) con y sin RAG.

In [16]:
# Usamos primero una muestra de DEV para ajustar
dev_data = datasets["dev"].head(10)

print("="*70)
print("EVALUANDO GPT-3.5 TURBO SIN RAG")
print("="*70)
system_gpt_no_rag = GuaraniTransformationSystem(model_gpt35, retriever=None)
results_gpt_no_rag = system_gpt_no_rag.evaluate_dataset(dev_data, use_rag=False)

print("\n" + "="*70)
print("EVALUANDO GPT-3.5 TURBO CON RAG")
print("="*70)
system_gpt_rag = GuaraniTransformationSystem(model_gpt35, retriever=retriever)
results_gpt_rag = system_gpt_rag.evaluate_dataset(dev_data, use_rag=True)

print("\n" + "="*70)
print("EVALUANDO CLAUDE 3.5 SONNET SIN RAG")
print("="*70)
system_claude_no_rag = GuaraniTransformationSystem(model_claude, retriever=None)
results_claude_no_rag = system_claude_no_rag.evaluate_dataset(dev_data, use_rag=False)

print("\n" + "="*70)
print("EVALUANDO CLAUDE 3.5 SONNET CON RAG")
print("="*70)
system_claude_rag = GuaraniTransformationSystem(model_claude, retriever=retriever)
results_claude_rag = system_claude_rag.evaluate_dataset(dev_data, use_rag=True)

print("\n✓ Evaluación en DEV completada (10 ejemplos)")


EVALUANDO GPT-3.5 TURBO SIN RAG

EVALUANDO GPT-3.5 TURBO CON RAG

EVALUANDO CLAUDE 3.5 SONNET SIN RAG

EVALUANDO CLAUDE 3.5 SONNET CON RAG

✓ Evaluación en DEV completada (10 ejemplos)


## 10. Cálculo de Métricas

In [17]:
def calculate_metrics(results: List[Dict]) -> Dict:
    """Calcula métricas de evaluación (Accuracy + BLEU)"""

    total = len(results)
    correct = sum(1 for r in results if r["correct"])
    accuracy = (correct / total) * 100 if total > 0 else 0

    # Limpiar predicciones para evitar errores del BLEU
    preds = [str(r["prediction"]).strip() for r in results]
    refs  = [[str(r["target"]).strip()] for r in results]

    bleu = BLEU()
    try:
        bleu_score = bleu.corpus_score(preds, refs).score
    except:
        bleu_score = 0.0

    return {
        "total": total,
        "correct": correct,
        "accuracy": accuracy,
        "bleu": bleu_score
    }


# Calcular métricas para cada configuración
metrics = {
    "GPT-3.5 Sin RAG": calculate_metrics(results_gpt_no_rag),
    "GPT-3.5 Con RAG": calculate_metrics(results_gpt_rag),
    "Claude 3.5 Sin RAG": calculate_metrics(results_claude_no_rag),
    "Claude 3.5 Con RAG": calculate_metrics(results_claude_rag)
}

# Mostrar resultados
print("\n" + "="*70)
print("RESULTADOS DE EVALUACIÓN")
print("="*70)

for config, m in metrics.items():
    print(f"\n{config}:")
    print(f"  Accuracy: {m['accuracy']:.2f}%")
    print(f"  BLEU Score: {m['bleu']:.2f}")
    print(f"  Correctas: {m['correct']}/{m['total']}")



RESULTADOS DE EVALUACIÓN

GPT-3.5 Sin RAG:
  Accuracy: 0.00%
  BLEU Score: 19.00
  Correctas: 0/10

GPT-3.5 Con RAG:
  Accuracy: 0.00%
  BLEU Score: 15.97
  Correctas: 0/10

Claude 3.5 Sin RAG:
  Accuracy: 20.00%
  BLEU Score: 0.00
  Correctas: 2/10

Claude 3.5 Con RAG:
  Accuracy: 20.00%
  BLEU Score: 0.00
  Correctas: 2/10


## 11. Tabla Comparativa

In [18]:
# Crear DataFrame comparativo
comparison_df = pd.DataFrame({
    "Modelo": ["GPT-3.5 Turbo", "GPT-3.5 Turbo", "Claude 3.5 Sonnet", "Claude 3.5 Sonnet"],
    "Estrategia": ["Sin RAG", "Con RAG", "Sin RAG", "Con RAG"],
    "Accuracy (%)": [
        metrics["GPT-3.5 Sin RAG"]["accuracy"],
        metrics["GPT-3.5 Con RAG"]["accuracy"],
        metrics["Claude 3.5 Sin RAG"]["accuracy"],
        metrics["Claude 3.5 Con RAG"]["accuracy"]
    ],
    "BLEU": [
        metrics["GPT-3.5 Sin RAG"]["bleu"],
        metrics["GPT-3.5 Con RAG"]["bleu"],
        metrics["Claude 3.5 Sin RAG"]["bleu"],
        metrics["Claude 3.5 Con RAG"]["bleu"]
    ]
})

print("\n" + "="*70)
print("TABLA COMPARATIVA")
print("="*70)
print(comparison_df.to_string(index=False))

# Encontrar mejor configuración por Accuracy
best_idx = comparison_df["Accuracy (%)"].idxmax()
best_config = comparison_df.iloc[best_idx]

print("\n" + "="*70)
print("MEJOR CONFIGURACIÓN (por Accuracy)")
print("="*70)
print(f"Modelo: {best_config['Modelo']}")
print(f"Estrategia: {best_config['Estrategia']}")
print(f"Accuracy: {best_config['Accuracy (%)']:.2f}%")
print(f"BLEU: {best_config['BLEU']:.2f}")



TABLA COMPARATIVA
           Modelo Estrategia  Accuracy (%)      BLEU
    GPT-3.5 Turbo    Sin RAG           0.0 18.995892
    GPT-3.5 Turbo    Con RAG           0.0 15.973578
Claude 3.5 Sonnet    Sin RAG          20.0  0.000000
Claude 3.5 Sonnet    Con RAG          20.0  0.000000

MEJOR CONFIGURACIÓN (por Accuracy)
Modelo: Claude 3.5 Sonnet
Estrategia: Sin RAG
Accuracy: 20.00%
BLEU: 0.00


## 12. Ejemplos de Transformaciones

In [19]:
# Mostrar algunos ejemplos de transformaciones
print("\n" + "="*70)
print("EJEMPLOS DE TRANSFORMACIONES (GPT-3.5 CON RAG)")
print("="*70)

for i, result in enumerate(results_gpt_rag[:5], 1):
    print(f"\nEjemplo {i}:")
    print(f"  ID: {result['id']}")
    print(f"  Oración original: {result['source']}")
    print(f"  Transformación: {result['change']}")
    print(f"  Esperado: {result['target']}")
    print(f"  Generado: {result['prediction']}")
    print(f"  ✓ Correcto" if result['correct'] else "  ✗ Incorrecto")


EJEMPLOS DE TRANSFORMACIONES (GPT-3.5 CON RAG)

Ejemplo 1:
  ID: Guarani0232
  Oración original: Ore ndorombyai kuri
  Transformación: TYPE:AFF
  Esperado: Ore rombyai kuri
  Generado: Ore ndorombyai kuri'i.
  ✗ Incorrecto

Ejemplo 2:
  ID: Guarani0233
  Oración original: Ore ndorombyai kuri
  Transformación: TENSE:FUT_SIM
  Esperado: Ore ndorombyaita
  Generado: Ore ndorombyai kuri ojeporive.
  ✗ Incorrecto

Ejemplo 3:
  ID: Guarani0234
  Oración original: Ore ndorombyai kuri
  Transformación: PERSON:1_PL_INC
  Esperado: Ñande nañambyai kuri
  Generado: Ro-ndorombyai kuri.
  ✗ Incorrecto

Ejemplo 4:
  ID: Guarani0235
  Oración original: Ore ndorombyai kuri
  Transformación: PERSON:1_SI
  Esperado: Che nambyai kuri
  Generado: Añe'ẽse: Añe'ẽa ndorombyai kuri.
  ✗ Incorrecto

Ejemplo 5:
  ID: Guarani0236
  Oración original: Ore ndorombyai kuri
  Transformación: PERSON:2_PL
  Esperado: Peẽ napembyai kuri
  Generado: Pe ore ndorombyai kuripe.
  ✗ Incorrecto


## 13. Exportar Resultados

In [20]:
# Guardar resultados completos
all_results = {
    "gpt35_no_rag": results_gpt_no_rag,
    "gpt35_rag": results_gpt_rag,
    "claude_no_rag": results_claude_no_rag,
    "claude_rag": results_claude_rag,
    "metrics": metrics,
    "metadata": {
        "dataset": "AmericasNLP 2025 - Guaraní",
        "models": ["openai/gpt-3.5-turbo", "anthropic/claude-3.5-sonnet"],
        "strategies": ["Sin RAG", "Con RAG"],
        "embedding_model": "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
        "vector_store": "FAISS",
        "sources": ["Gramática guaraní.pdf", "Diccionario guaraní-español.pdf"]
    }
}

# Guardar resultados en JSON
with open("guarani_transformation_results.json", "w", encoding="utf-8") as f:
    json.dump(all_results, f, ensure_ascii=False, indent=2)

print("✓ Resultados guardados: guarani_transformation_results.json")

# Guardar tabla comparativa en CSV
comparison_df.to_csv("comparison_table.csv", index=False)
print("✓ Tabla comparativa guardada: comparison_table.csv")

# Descargar archivos (solo para Colab)
try:
    from google.colab import files
    files.download("guarani_transformation_results.json")
    files.download("comparison_table.csv")
    print("✓ Descarga completada")
except:
    pass


✓ Resultados guardados: guarani_transformation_results.json
✓ Tabla comparativa guardada: comparison_table.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ Descarga completada


# Conclusiones

## 1. ¿Qué modelo es mejor?

- Claude 3.5 Sonnet obtuvo el mejor accuracy (20%).
- GPT-3.5 Turbo no tuvo aciertos exactos (0%), pero alcanzó BLEU más alto (~19), indicando mayor similitud parcial.

**Conclusión:**  
Claude es mejor en exactitud; GPT-3.5 genera transformaciones más parecidas pero nunca exactas.

---

## 2. ¿RAG mejora el rendimiento?

- En GPT-3.5, RAG redujo el BLEU y no mejoró el accuracy.
- En Claude 3.5, RAG no produjo cambios en accuracy y BLEU quedó en 0.

**Conclusión:**  
RAG no mejoró el rendimiento debido a que las reglas del dataset no están bien representadas en los documentos usados para el vector store.

---

## 3. ¿Por qué se obtuvieron estos resultados?

- El guaraní es un idioma de bajo recurso.
- Los LLM generalistas no tienen entrenamiento profundo en su morfología.
- Las transformaciones requieren reglas específicas que no están explícitas en los PDFs.
- Claude maneja mejor las estructuras y logra algunos aciertos exactos.
- GPT-3.5 se acerca más en similitud pero falla en precisión exacta.
